READING FILE<PreProcessing>

In [141]:
import pandas as pd
import numpy as np 
df = pd.read_csv("hydrogen fuel data.csv")



C:\Users\darji\AppData\Local\Temp\ipykernel_26672\1771419806.py:3: DtypeWarning: Columns (13,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("hydrogen fuel data.csv")


In [ ]:
data_text = df[["Link Text"]]
data_text.loc[:, 'index'] = df.index
documents = data_text
pd.DataFrame(documents)
documents.tail()

In [144]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [ ]:
import nltk
import re
nltk.download('wordnet')

In [146]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(text)


In [147]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))


original document: 
['Photos', 'from', 'Ayr', 'State', 'High', "School's", 'post']


 tokenized and lemmatized document: 
['photo', 'state', 'high', 'school', 'post']


In [ ]:
processed_docs= documents.dropna(subset = ['Link Text'],inplace=True)



In [ ]:
processed_docs = documents['Link Text'].map(preprocess)
processed_docs[:10]

Dictonary of words


In [196]:
dictionary = gensim.corpora.Dictionary(processed_docs)

Dictionary<0 unique tokens: []>


In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [152]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [153]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(0, 1), (26, 1), (251, 1)]

In [154]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))


Word 0 ("photo") appears 1 time.
Word 26 ("post") appears 1 time.
Word 251 ("quantron") appears 1 time.


In [155]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [156]:
corpus_tfidf = tfidf[bow_corpus]

In [157]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5541679690417858), (1, 0.8324048666893427)]


In [158]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [159]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.092*"hydrogen" + 0.059*"fuel" + 0.031*"power" + 0.019*"truck" + 0.017*"green" + 0.017*"cell" + 0.014*"clean" + 0.013*"energi" + 0.012*"fossil" + 0.012*"news"
Topic: 1 
Words: 0.088*"hydrogen" + 0.046*"fuel" + 0.040*"electr" + 0.029*"vehicl" + 0.025*"cell" + 0.025*"power" + 0.015*"futur" + 0.015*"truck" + 0.014*"car" + 0.011*"massiv"
Topic: 2 
Words: 0.127*"hydrogen" + 0.080*"fuel" + 0.055*"cell" + 0.028*"power" + 0.021*"develop" + 0.013*"life" + 0.013*"produc" + 0.013*"plant" + 0.012*"crude" + 0.011*"india"
Topic: 3 
Words: 0.063*"fuel" + 0.063*"launch" + 0.055*"nasa" + 0.054*"hydrogen" + 0.043*"moon" + 0.043*"rocket" + 0.042*"leak" + 0.030*"attempt" + 0.023*"artemi" + 0.019*"power"
Topic: 4 
Words: 0.057*"hydrogen" + 0.028*"energi" + 0.027*"post" + 0.026*"photo" + 0.023*"green" + 0.017*"emiss" + 0.016*"power" + 0.015*"electr" + 0.014*"carbon" + 0.014*"zero"
Topic: 5 
Words: 0.096*"hydrogen" + 0.047*"power" + 0.035*"energi" + 0.029*"clean" + 0.028*"climat" + 0.023*"f

In [160]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [161]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.195*"timelin" + 0.150*"photo" + 0.032*"post" + 0.016*"share" + 0.016*"energi" + 0.015*"asia" + 0.015*"pimagazin" + 0.015*"bangladesh" + 0.014*"lead" + 0.014*"turbin"
Topic: 1 Word: 0.030*"fuel" + 0.026*"launch" + 0.024*"nasa" + 0.021*"rocket" + 0.021*"moon" + 0.020*"leak" + 0.018*"hydrogen" + 0.018*"cell" + 0.012*"futur" + 0.012*"ruin"
Topic: 2 Word: 0.022*"climat" + 0.022*"solut" + 0.021*"ocean" + 0.019*"research" + 0.018*"california" + 0.017*"power" + 0.017*"attempt" + 0.016*"hydrogen" + 0.014*"energi" + 0.012*"photo"
Topic: 3 Word: 0.021*"hydrogen" + 0.019*"fuel" + 0.015*"energi" + 0.012*"green" + 0.012*"power" + 0.010*"fossil" + 0.010*"engin" + 0.009*"timelin" + 0.009*"cell" + 0.008*"batteri"
Topic: 4 Word: 0.022*"power" + 0.021*"post" + 0.020*"photo" + 0.017*"hydrogen" + 0.013*"fuel" + 0.012*"energi" + 0.010*"green" + 0.010*"plan" + 0.010*"test" + 0.009*"news"
Topic: 5 Word: 0.024*"hydrogen" + 0.018*"green" + 0.015*"fuel" + 0.015*"cell" + 0.014*"power" + 0.013*"en

In [162]:
processed_docs[4310]

['photo', 'state', 'high', 'school', 'post']

In [163]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7749659419059753	 
Topic: 0.213*"photo" + 0.118*"timelin" + 0.098*"post" + 0.048*"hydrogen" + 0.038*"energi" + 0.026*"fuel" + 0.024*"transit" + 0.020*"turbin" + 0.020*"lead" + 0.019*"asia"

Score: 0.025012921541929245	 
Topic: 0.092*"hydrogen" + 0.059*"fuel" + 0.031*"power" + 0.019*"truck" + 0.017*"green" + 0.017*"cell" + 0.014*"clean" + 0.013*"energi" + 0.012*"fossil" + 0.012*"news"

Score: 0.0250038281083107	 
Topic: 0.063*"fuel" + 0.063*"launch" + 0.055*"nasa" + 0.054*"hydrogen" + 0.043*"moon" + 0.043*"rocket" + 0.042*"leak" + 0.030*"attempt" + 0.023*"artemi" + 0.019*"power"

Score: 0.025003353133797646	 
Topic: 0.057*"hydrogen" + 0.028*"energi" + 0.027*"post" + 0.026*"photo" + 0.023*"green" + 0.017*"emiss" + 0.016*"power" + 0.015*"electr" + 0.014*"carbon" + 0.014*"zero"

Score: 0.025002496317029	 
Topic: 0.084*"hydrogen" + 0.046*"fuel" + 0.026*"cell" + 0.025*"toyota" + 0.024*"power" + 0.019*"test" + 0.018*"vehicl" + 0.018*"work" + 0.015*"share" + 0.015*"energi"

Score: 0.

In [164]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7749714851379395	 
Topic: 0.098*"post" + 0.083*"photo" + 0.015*"hydrogen" + 0.012*"hyundai" + 0.011*"fuel" + 0.011*"power" + 0.011*"citi" + 0.010*"energi" + 0.009*"green" + 0.009*"cell"

Score: 0.025010421872138977	 
Topic: 0.195*"timelin" + 0.150*"photo" + 0.032*"post" + 0.016*"share" + 0.016*"energi" + 0.015*"asia" + 0.015*"pimagazin" + 0.015*"bangladesh" + 0.014*"lead" + 0.014*"turbin"

Score: 0.025003336369991302	 
Topic: 0.019*"hydrogen" + 0.017*"power" + 0.017*"green" + 0.016*"energi" + 0.014*"fuel" + 0.013*"clean" + 0.012*"sign" + 0.011*"develop" + 0.009*"deal" + 0.009*"ammonia"

Score: 0.025003068149089813	 
Topic: 0.029*"hydrogen" + 0.017*"timelin" + 0.017*"fuel" + 0.016*"cell" + 0.015*"photo" + 0.014*"power" + 0.014*"green" + 0.012*"plant" + 0.012*"news" + 0.012*"life"

Score: 0.02500264346599579	 
Topic: 0.030*"fuel" + 0.026*"launch" + 0.024*"nasa" + 0.021*"rocket" + 0.021*"moon" + 0.020*"leak" + 0.018*"hydrogen" + 0.018*"cell" + 0.012*"futur" + 0.012*"ruin"

Score

In [165]:
unseen_document = input()
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.36961957812309265	 Topic: 0.092*"hydrogen" + 0.059*"fuel" + 0.031*"power" + 0.019*"truck" + 0.017*"green"
Score: 0.21131421625614166	 Topic: 0.127*"hydrogen" + 0.080*"fuel" + 0.055*"cell" + 0.028*"power" + 0.021*"develop"
Score: 0.17109526693820953	 Topic: 0.096*"hydrogen" + 0.047*"power" + 0.035*"energi" + 0.029*"clean" + 0.028*"climat"
Score: 0.0901322290301323	 Topic: 0.088*"hydrogen" + 0.046*"fuel" + 0.040*"electr" + 0.029*"vehicl" + 0.025*"cell"
Score: 0.06326694041490555	 Topic: 0.073*"hydrogen" + 0.043*"green" + 0.042*"power" + 0.031*"energi" + 0.027*"renew"
Score: 0.058922797441482544	 Topic: 0.084*"hydrogen" + 0.046*"fuel" + 0.026*"cell" + 0.025*"toyota" + 0.024*"power"
Score: 0.03422674536705017	 Topic: 0.085*"hydrogen" + 0.072*"green" + 0.032*"project" + 0.031*"energi" + 0.024*"power"


In [187]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs,dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)


Coherence Score:  0.32472290323498537


In [167]:
coherence_model_TFIDF = CoherenceModel(model=lda_model_tfidf,texts=processed_docs,coherence='c_v')
coherence_model_TFIDF = coherence_model_TFIDF.get_coherence()
print('coherence Score:',coherence_model_TFIDF)

coherence Score: 0.35174063669427047


In [189]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [190]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(bow_corpus)
corpus_sets = [gensim.utils.ClippedCorpus(bow_corpus, int(num_of_docs*0.75)), 
               bow_corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i],dictionary=dictionary,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)

                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close() 
     

100%|██████████| 540/540 [2:03:00<00:00, 13.67s/it]


In [ ]:
num_topics = 8

lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [1]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./results/ldavis_tuned_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_tuned_'+ str(num_topics) +'.html')

LDAvis_prepared

NameError: name 'os' is not defined